In [ ]:
# Data Extraced from CNN, FOX and MSNBC folders from the files in the local computer 
# (The original data files are really large to be uploaded along with the project files)
channel = "CNN"

party_words = ['biden', 'joe', 'joebiden', 'donald', 'trump', 'donaldtrump']
              
keywords = ['win', 'won', 'vote', 'winner']

def find_word(text):
    py_stem = PorterStemmer ()
    token = [py_stem.stem(word) for word in text.split()]
    if any(word in token for word in party_words) and any(word in token for word in keywords): #checking if any one of the keyword matches the string
        return True
    return False

dir_name = f"{channel}_merged/{channel}_parsed/"
comment_set = []

cnt = 0 
st = time.time()
for file in os.listdir(dir_name+"metadata/"):
    video_id = None
    if file.endswith(".csv"):
        video_id = file[:-len("_metadata.csv")]
        # print(video_id)

    if video_id is None: continue
    
    with open(dir_name+"jsonbyline/"+video_id+"_jsonbyline.txt", "r", encoding='utf-8') as fi:
        ids, text = [], []
        # print(video_id)
        for line in fi:
            cnt+=1
            # x = None
            if ", 'text': " in line:

                dictionary = ast.literal_eval(line)
                dictionary['text'] = dictionary['text'].encode('unicode-escape').decode('ascii')
                #print(dictionary)
                x = dictionary['commentId']
                #print(x)
                y = dictionary['text']
                #print(y)
                if find_word(y):
                    ids.append(x)
                    text.append(y)

            else:

                dictionary = ast.literal_eval(line)
                dictionary['text'] = dictionary['text'].encode('unicode-escape').decode('ascii')
                #print(dictionary)
                x = dictionary['commentId']
                #print(x)
                y = dictionary['text']
                #print(y)
                if find_word(y):
                    ids.append(x)
                    text.append(y)
        df = pd.DataFrame()
        df["commentId"] = ids
        df["text"] = text
        df = df.drop_duplicates(subset=["commentId"], keep='first')
        df["commentId"] =  df["commentId"].astype(str)

        _df = pd.read_csv(dir_name+"metadata/"+video_id+"_metadata.csv")
        _df = _df.drop_duplicates(subset=["commentId"], keep='first')
        _df["commentId"] =  _df["commentId"].astype(str)
        
        _df["timeline"] = _df["publishedAt"].str.slice(stop=10)
        _df["timeline"] = pd.to_datetime(_df["timeline"], format="%Y-%m-%d")
        _df = _df[(_df["timeline"] > '2020-10-19') & (_df["timeline"] < '2020-11-08')]
        df = df.join(_df.set_index("commentId"), on="commentId")
        
        df = df[["videoId", "commentId", "text", "likeCount", "totalReplyCount", "timeline"]]
        
        # if video_id in big_data:
        #     df["videoDate"] = big_data[video_id]["date"]
        # else: df["videoDate"] = -1

        # remove duplicate comments
        
        df.to_csv(f'forecasting_{channel}.csv', mode='a', index=False, header=not os.path.exists(f'forecasting_{channel}.csv'))
en = time.time()
print("first pass: ", en-st)
print(cnt)